In [44]:

import collections
import math
from typing import List, Tuple, Dict
import re
import time

def SimplificarSTR(input:str):
      return "<s>" if input == "<s>" else "</s>" if input == "</s>" else re.sub( r"[^a-z0-9 ]+","", input.lower(),)



## PREPROCESAR CORPUS

In [2]:
import corpusador as corpu
corpu.CorpusProcesador(corpu.getSmallCorpus(), "data/corpus_1")      
corpu.CorpusProcesador(corpu.getMedCorpus(), "data/corpus_2")
#corpu.CorpusProcesador(corpu.getBigCorpus(), "data/corpus_3") #CORPUS MUY PESADO SE EXPANDEA A 1.24GB
#BAJAR DE https://www.kaggle.com/datasets/jannesklaas/scifi-stories-text-corpus y 
#agregar a la carpeta como big_corpus.txt

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 esimo ngrama con 12384 tokens
2 esimo ngrama con 98965 tokens
3 esimo ngrama con 166844 tokens
4 esimo ngrama con 190525 tokens
5 esimo ngrama con 195613 tokens
1 esimo ngrama con 24716 tokens
2 esimo ngrama con 156255 tokens
3 esimo ngrama con 275282 tokens
4 esimo ngrama con 314913 tokens
5 esimo ngrama con 323987 tokens
1 esimo ngrama con 286776 tokens
2 esimo ngrama con 5090206 tokens
3 esimo ngrama con 14777418 tokens
4 esimo ngrama con 21887428 tokens
5 esimo ngrama con 24567886 tokens


* Formula para k-grama:
$$P_{KN}(W_{n}|W_{n-1}...W_{n-k}) = \frac{max(C(W_{n-k}...W_{n}) - d, 0)}{W_{n-k}...W_{n - 1}} + \lambda(W_{n-k}..W_{n-1})P_{KN}(W_{n}|W_{n-1}...W_{n-k + 1})$$

In [187]:
class Ngramador:
    D = 0.75
    def __init__(self, dir_corpus, n = 3):
        self.nN = n
        #tokenized_corpus = self.tokenize_corpus(corpus)
        #self.lstNGram = [self.count_ngrams(tokenized_corpus, i + 1) for i in range(self.nN)]
        #self.continuation_counts = self.compute_unique_context_counts(self.lstNGram[1])
        #self.total_continuations = len(self.lstNGram[1])
        self.dic, self.revdic = self.LeerDiccionario(dir_corpus)
        self.recuerdos = {}
        self.lstNGram = []
        for i in range(self.nN):
            self.lstNGram.append(self.LeerNgrama(dir_corpus, i))
            print(f"{i+1} esimo ngrama con {len(self.lstNGram[i])} tokens")
            
        self.continuation_counts = self.compute_unique_context_counts(self.lstNGram[1])
        self.total_continuations = len(self.lstNGram[1])
        
        
    def LeerDiccionario(self, dir_corpus):
        file = open(dir_corpus + "/dic.txt", "r")
        lines = file.readlines()
        dic = {}
        rev_dic = {}
        
        for line in lines:
            sp = line.replace("\n", "").split(":")
            dic[sp[0]] = int(sp[1])
            rev_dic[int(sp[1])] = sp[0]
        file.close()
        return dic, rev_dic
    def LeerNgrama(self, dir_corpus, i):
        t0 = round(time.time() * 1000)
        file = open(dir_corpus + f"/NG{i + 1}.txt", "r")
        lines = file.readlines()
        file.close()
        ngram_counts = {}
        t1 = round(time.time() * 1000)
        for line in lines:
            sp = line.split(":")
            sp0 = sp[0].split()
            sp0 = tuple([int(num) for num in sp0])
            sp1 = int(sp[1].replace("\n", ""))
            ngram_counts[sp0] = sp1     
        t2= round(time.time() * 1000)
        print(f"Tiempos 0-1 : {t1 - t0}ms - Tiempos 1-2 : {t2 - t1}ms")
        return ngram_counts
      
    def tokenize_corpus(self, corpus: List[str]) -> List[List[str]]: 
        return [ ['<s>'] + sentence.lower().split() + ['</s>'] for sentence in corpus]

    def count_ngrams(self, tokenized_corpus: List[List[str]], n: int) -> Dict[Tuple[str, ...], int]:
        ngram_counts = collections.Counter()
        for tokens in tokenized_corpus:
            for i in range(len(tokens) - n + 1):
                ngram = tuple(tokens[i:i + n])
                ngram_counts[ngram] += 1
        return ngram_counts
    
    #Cálculo de la probabilidad de continuación
    def continuation_probability(self, word: str, continuation_counts: Dict[str, int], total_continuations: int) -> float:
        return continuation_counts[word] / total_continuations

    #Cálculo del factor de normalización
    #context NGRAMA (P1,P2,P3...)
    #ngram_counts NGRAMAS
    def flambda(self, context: Tuple[str, ...], ngram_counts: Dict[Tuple[str, ...], int], lower_order_counts: Dict[Tuple[str, ...], int]) -> float:
        #context_count = sum([count for ngram, count in ngram_counts.items() if ngram[:-1] == context])
        #mcontext = lower_order_counts[context]
        #print(f"context_count: {context_count} = {mcontext}")
        context_count = lower_order_counts[context]
        if (context in self.recuerdos.keys()):
            sum = self.recuerdos[context]
        else:
            sum = 0
            for ngram in ngram_counts.keys():
                sum += 1 if ngram[:-1] == context else 0
            self.recuerdos[context] = sum
        #unique_continuations = len([1 for ngram in ngram_counts if ngram[:-1] == context])
        #munique = sum
        unique_continuations = sum
        #print(f"unique_continuations: {unique_continuations} = {munique}")
        return (Ngramador.D * unique_continuations) / context_count if context_count > 0 else 0
    
    #Cálculo de conteos de contextos unicos (para probabilidad de continuación)
    def compute_unique_context_counts(self, bigram_counts: Dict[Tuple[str, str], int]) -> Dict[str, int]:
        continuation_counts = collections.Counter()
        for (w_prev, w_next) in bigram_counts:
            continuation_counts[w_next] += 1
        return continuation_counts

    def destraducir(self, tup):
        return tuple([self.revdic[t] for t in tup])
    
    def PKN(self, ngram: Tuple):
        N = len(ngram) - 1
        
        NGN_count = self.lstNGram[N].get(ngram, 0)
        NGNl_count = self.lstNGram[N - 1].get(ngram[:-1], 0)
        #print(f"{N}-> NG{ngram} Buscando : {ngram[:-1]} = {self.destraducir(ngram[:-1])} EN NG{N} ----> {NGNl_count}")
        #print(f"TIPOS : NGN {type(ngram[:-1][0])} - NGN_LST {type(list(self.lstNGram[N - 1].keys())[0][0])}")
        if(N == 0): 
            res = self.continuation_probability(ngram[0], self.continuation_counts, self.total_continuations)
            #print(f"RES N1 : {res}")
            return res
        if NGNl_count > 0:
            
            t0 = round(time.time() * 1000)
            #print(f"ngl = {ngram[:-1]}, con N = {len(list(self.lstNGram[N - 1].keys())[0])}")
            fl = self.flambda(ngram[:-1], self.lstNGram[N], self.lstNGram[N - 1])
            t1 = round(time.time() * 1000)
            #print(f"flambda :  analizado para {ngram[:-1]} con {t1 - t0}ms")
            #print(f"PKN{len(ngram)} - IFEA LLAMADO")
            res = max(NGN_count - Ngramador.D, 0) / NGNl_count + fl * self.PKN(ngram[1:])
            #t1 = round(time.time() * 1000)
        else:
            #print(f"PKN{len(ngram)} - ELSE LLAMADO")
            res = self.PKN(ngram[1:])
            #t1 = round(time.time() * 1000)
        #print(f"RES : {res}")
        #t2 = round(time.time() * 1000)
        
        return res
    
    def sentence_probability(self, sentence: str):
        
        tokens = ['<s>'] + sentence.lower().split() + ['</s>']
        print(f"N-Grama : {tokens}")
        tokens = [int(self.dic.get(SimplificarSTR(t), -1)) for t in tokens]
        print(f"N-Grama traducido: {tokens}")
        
        sum = 0.0
        
        for i in range(len(tokens) - self.nN + 1):
            t0 = round(time.time() * 1000)
            ngrama = tuple([tokens[i + k] for k in range(self.nN)])
            prob = self.PKN(ngrama)
            t1 = round(time.time() * 1000)
            #print(f"N-Grama : {ngrama} analizado con {t1 - t0}ms")
            #print(prob)
            sum += math.log2(prob) if prob > 0 else float('-inf')
            #probability_log_sum += math.log(prob) if prob > 0 else float('-inf')
            
        #return math.exp(sum)
        return 2**(-1 / len(tokens) * sum)

## CORPUS PEQUEÑO

In [189]:
dir_corpus = "data/corpus_1"
ngdor = Ngramador(dir_corpus, n= 5)
print("------------------------------------------------")
test = ["of their substitute for speech in a singular method", 
        "new york is to big", 
        "york town is to big",
        "this current acquired a monstrous velocity",
        "el pollito pio el pollito pio"]

for t in test:
      p = ngdor.sentence_probability(t)
      print(f"La perplejidad de la oración '{t}' es: {p:.4n}")
      print("------------------------------------------------")

Tiempos 0-1 : 1ms - Tiempos 1-2 : 7ms
1 esimo ngrama con 12384 tokens
Tiempos 0-1 : 9ms - Tiempos 1-2 : 71ms
2 esimo ngrama con 98965 tokens
Tiempos 0-1 : 15ms - Tiempos 1-2 : 133ms
3 esimo ngrama con 166844 tokens
Tiempos 0-1 : 19ms - Tiempos 1-2 : 166ms
4 esimo ngrama con 190525 tokens
Tiempos 0-1 : 16ms - Tiempos 1-2 : 177ms
5 esimo ngrama con 195613 tokens
------------------------------------------------
N-Grama : ['<s>', 'of', 'their', 'substitute', 'for', 'speech', 'in', 'a', 'singular', 'method', '</s>']
N-Grama traducido: [12382, 48, 764, 5901, 181, 2906, 21, 13, 7727, 4382, 12383]
La perplejidad de la oración 'of their substitute for speech in a singular method' es: 45.66
------------------------------------------------
N-Grama : ['<s>', 'new', 'york', 'is', 'to', 'big', '</s>']
N-Grama traducido: [12382, 665, 1692, 3, 85, 2693, 12383]
La perplejidad de la oración 'new york is to big' es: 17.27
------------------------------------------------
N-Grama : ['<s>', 'york', 'town', 

## CORPUS MEDIANO

In [188]:
dir_corpus = "data/corpus_2"
ngdor = Ngramador(dir_corpus, n= 3)
print("------------------------------------------------")
test = ["of their substitute for speech in a singular method", 
        "new york is to big", 
        "york town is to big",
        "this current acquired a monstrous velocity",
        "a big tower in the sea"]

for t in test:
      p = ngdor.sentence_probability(t)
      print(f"La perplejidad de la oración '{t}' es: {p:.4n}")
      print("------------------------------------------------")

Tiempos 0-1 : 2ms - Tiempos 1-2 : 15ms
1 esimo ngrama con 24716 tokens
Tiempos 0-1 : 15ms - Tiempos 1-2 : 118ms
2 esimo ngrama con 156255 tokens
Tiempos 0-1 : 23ms - Tiempos 1-2 : 197ms
3 esimo ngrama con 275282 tokens
------------------------------------------------
N-Grama : ['<s>', 'of', 'their', 'substitute', 'for', 'speech', 'in', 'a', 'singular', 'method', '</s>']
N-Grama traducido: [24714, 3, 623, 13505, 44, 2185, 5, 29, 1084, 6197, 24715]
La perplejidad de la oración 'of their substitute for speech in a singular method' es: 27.8
------------------------------------------------
N-Grama : ['<s>', 'new', 'york', 'is', 'to', 'big', '</s>']
N-Grama traducido: [24714, 1302, 7212, 256, 20, 4638, 24715]
La perplejidad de la oración 'new york is to big' es: 91.39
------------------------------------------------
N-Grama : ['<s>', 'york', 'town', 'is', 'to', 'big', '</s>']
N-Grama traducido: [24714, 7212, 3221, 256, 20, 4638, 24715]
La perplejidad de la oración 'york town is to big' es: 2

## CORPUS GRANDE

In [181]:
dir_corpus = "data/corpus_3"
ngdor = Ngramador(dir_corpus, n= 3)
print("------------------------------------------------")
test = ["of their substitute for speech in a singular method", 
        "new york is to big", 
        "york town is to big",
        "this current acquired a monstrous velocity"]

for t in test:
      p = ngdor.sentence_probability(t)
      print(f"La perplejidad de la oración '{t}' es: {p:.4n}")
      print("------------------------------------------------")

Tiempos 0-1 : 21ms - Tiempos 1-2 : 126ms
1 esimo ngrama con 286776 tokens
Tiempos 0-1 : 411ms - Tiempos 1-2 : 3520ms
2 esimo ngrama con 5090206 tokens
Tiempos 0-1 : 1165ms - Tiempos 1-2 : 12793ms
3 esimo ngrama con 14777418 tokens
------------------------------------------------
N-Grama : ['<s>', 'of', 'their', 'substitute', 'for', 'speech', 'in', 'a', 'singular', 'method', '</s>']
N-Grama traducido: [286774, 46, 120, 7515, 24, 10757, 37, 60, 37062, 1685, 286775]
La perplejidad de la oración 'of their substitute for speech in a singular method' es: 952.5
------------------------------------------------
N-Grama : ['<s>', 'new', 'york', 'is', 'to', 'big', '</s>']
N-Grama traducido: [286774, 4, 19, 10, 55, 983, 286775]
La perplejidad de la oración 'new york is to big' es: 295.3
------------------------------------------------
N-Grama : ['<s>', 'york', 'town', 'is', 'to', 'big', '</s>']
N-Grama traducido: [286774, 19, 2004, 10, 55, 983, 286775]
La perplejidad de la oración 'york town is to